# OpenAI Agent with LlamaIndex

## Install Dependencies

In [104]:
!pip install uv
!uv pip install -qU llama-index==0.11.6 llama-index-llms-openai llama-index-readers-file llama-index-embeddings-openai "openinference-instrumentation-llama-index>=2" arize-phoenix python-dotenv
!uv pip install -q -U google-genai
!uv pip install weasyprint

I0000 00:00:1750538644.349066   13377 fork_posix.cc:71] Other threads are currently calling into gRPC, skipping fork() handlers
10417.35s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


I0000 00:00:1750538650.433569   13377 fork_posix.cc:71] Other threads are currently calling into gRPC, skipping fork() handlers
10423.42s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
I0000 00:00:1750538656.032513   13377 fork_posix.cc:71] Other threads are currently calling into gRPC, skipping fork() handlers
10429.03s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
I0000 00:00:1750538661.460151   13377 fork_posix.cc:71] Other threads are currently calling into gRPC, skipping fork() handlers
10434.44s - pydevd: Sending message related to process being replaced timed-out after 5 seconds


Using Python 3.12.3 environment at: /home/lilbluey/repos/20k-project/.venv
Resolved 13 packages in 189ms                                        
⠙ Preparing packages... (0/8)                                                   
⠙ Preparing packages... (0/8)--------------     0 B/38.86 KiB           
⠙ Preparing packages... (0/8)-------------- 14.91 KiB/38.86 KiB         
⠙ Preparing packages... (0/8)---------- 30.91 KiB/38.86 KiB         
⠙ Preparing packages... (0/8)---------- 38.86 KiB/38.86 KiB         
⠙ Preparing packages... (0/8)---------- 38.86 KiB/38.86 KiB         
cssselect2           ------------------------------     0 B/15.09 KiB
⠙ Preparing packages... (0/8)---------- 38.86 KiB/38.86 KiB         
cssselect2           ------------------------------ 14.91 KiB/15.09 KiB
⠙ Preparing packages... (0/8)---------- 38.86 KiB/38.86 KiB         
cssselect2           ------------------------------ 14.91 KiB/15.09 KiB
⠙ Preparing packages... (0/8)---------- 38.86 KiB/38.86 KiB         


## Setup API Keys


In [93]:
from os import environ
from dotenv import load_dotenv

load_dotenv()

GEMINI_API_KEY = environ["GEMINI_API_KEY"]

## Import libraries and setup LlamaIndex

In [94]:
from llama_index.core import (
    SimpleDirectoryReader,
    VectorStoreIndex,
    StorageContext,
    load_index_from_storage,
)
from llama_index.core.tools import QueryEngineTool, ToolMetadata
from llama_index.core.agent import ReActAgent
from llama_index.llms.openai import OpenAI

#used for getting folder and file path for resume.
from pathlib import Path 

# # Create an llm object to use for the QueryEngine and the ReActAgent
# llm = OpenAI(model="gpt-4")
from google import genai
from google.genai import types
import pathlib
import httpx

import builtins
import os
import pyppeteer
import asyncio
#from asyncio import py
from pyppeteer import launch
from pyppeteer import chromium_downloader

#Gets the resume file in the input resume directory

In [95]:
client = genai.Client(api_key=GEMINI_API_KEY)

def GetFirstFile(directory_path):
    try:
        resume_folder = pathlib.Path(directory_path)
        for resume in resume_folder.iterdir():
            #only look for pdf files.
            if resume.is_file() and resume.suffix == ".pdf":
                return resume.absolute()
        return None
    except FileNotFoundError:
        return None

def GetFileData(file_path):
    return file_path.read_bytes()
    

resume_path = GetFirstFile("../InputResume")
if resume_path == None : 
    print("Add your resume in pdf format into InputResume folder")
    exit()
resume_data = GetFileData(resume_path)


#Sends the resume to Gemini (TODO: Testing)#

In [121]:
job_posting_url = "https://www.indeed.com/m/viewjob?jk=db1e2083edec1315&from=serp&tk=1iua4tf59gifh811&advn=5908654242980451&adid=447065164&ad=-6NYlbfkN0Cc7GM2uq9-_va6doAzIGZAjomm4_8ikDeWx2s3NNOqO6lrkyYoafl5oGCB5hCQu8t2YycwQrkR6Rm9PwUEwsE-S5SH-lpa2BADtsiSIxtmOT0ryJWWishe3FCvxE76BB5vq7bd-yS-RqGGpJp7pnBVEPREr2w4fijMplbi6qD2SQb7GCFyF3vVMVvytBwgf2t_UIi3M_eVAQp9cBG0dWY3JwTyPz284HuuepQ91N0rooywNJ-J3FAxpsBUu_IDwqZfmEeTDO87RsNd3HMFhIvL90kWT0AW6o0i1-Wmnz2s0QAgs0jOvuQoGD9bUd-dnGKaBkaDoCE4b6bz4KS6yNTSO03iA6_PABqLlxsHoBe9BdD84k3yhOEeDgJqESjY2BOrMqLoN6PAnfEbYR7GJ3k0vlgKkQzlSzBem6U48-Nv-SmzNdLCLQNexI32Eal6gpABWOgD7Hm-wkIhYBPO738UhKL0xgbevpb7UOm-_1fx1LzJf392_3jOlZ_cxk2LwkM-NoEu5wpIQAldau4Q15IuJE6djl-GV0QZRWQojv6fZrVpifsBRiP1AxcwvHjbwkksNVH-c7UjiI9i_kanPBsP9lqGP022-g39PTh1Qa7O6v2gjrkbnoFztJ4E8R8HRCzT4LTxHGSUffKFFQEg88a-CyMnJGOsOIW3Qp4noSl5fVK7II-AVPyPgn1t_BjBAlKWkKpNnnyiPA%3D%3D&xkcb=SoBS6_M3wKB6N2y-gp0LbzkdCdPP"
prompt = "Finetune this resume for a programming job, and only return a html file. The resume should only be in black and white, following a professional format, and only one page long. Tailor the resume for this job posting " + job_posting_url
response = client.models.generate_content(
  model="gemini-2.5-flash",
  contents=[
      types.Part.from_bytes(
        data=resume_data,
        mime_type='application/pdf',
      ),
      prompt])

resume_html = response.text
first_bracket = resume_html.find('<')
last_bracket = resume_html.rfind('>')
if first_bracket != -1 and last_bracket != -1 and first_bracket < last_bracket :
    resume_html = resume_html[first_bracket : last_bracket+1]
    print(resume_html)

<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Joel Lee Jie - Software Engineer Resume</title>
    <style>
        body {
            font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
            margin: 0;
            padding: 20px;
            color: #333;
            background-color: #fff;
            max-width: 800px; /* Standard resume width for viewing */
            margin: 20px auto; /* Center on page */
            box-shadow: 0 0 10px rgba(0,0,0,0.1); /* Subtle shadow for definition */
            line-height: 1.5;
        }
        h1, h2, h3 {
            color: #000;
            margin-top: 0;
            margin-bottom: 5px;
        }
        h1 {
            font-size: 2.2em;
            text-align: center;
            margin-bottom: 10px;
        }
        h2 {
            font-size: 1.2em;
            border-bottom: 1px solid #ccc;
            padding-b

In [ ]:
def create_directory_if_not_exists(path):
    try:
        os.makedirs(path, exist_ok=True)
        print(f"Directory '{path}' created or already exists.")
    except OSError as e:
        print(f"Error creating directory '{path}': {e}")

# def WriteToFile(directory_path, file_name, file_data):
#     with builtins.open(directory_path + "/" + file_name, 'w', encoding='utf-8') as f:
#       f.write(response.text)
#     print("Success")

create_directory_if_not_exists("../OutputResume")


Directory '../OutputResume' created or already exists.


In [123]:
import weasyprint
weasyprint.HTML(string = resume_html).write_pdf("../OutputResume/UpdatedResume.pdf")

# Set up Phoenix

In [22]:
import phoenix as px
session = px.launch_app()

Existing running Phoenix instance detected! Shutting it down and starting a new instance...


🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📖 For more information on how to use Phoenix, check out https://arize.com/docs/phoenix


In [24]:
from openinference.instrumentation.llama_index import LlamaIndexInstrumentor
from phoenix.otel import register

tracer_provider = register()
LlamaIndexInstrumentor().instrument(tracer_provider=tracer_provider)

Overriding of current TracerProvider is not allowed


🔭 OpenTelemetry Tracing Details 🔭
|  Phoenix Project: default
|  Span Processor: SimpleSpanProcessor
|  Collector Endpoint: localhost:4317
|  Transport: gRPC
|  Transport Headers: {'user-agent': '****'}
|  
|  Using a default SpanProcessor. `add_span_processor` will overwrite this default.
|  
|  ⚠️ WARNING: It is strongly advised to use a BatchSpanProcessor in production environments.
|  
|  `register` has set this TracerProvider as the global OpenTelemetry default.
|  To disable this behavior, call `register` with `set_global_tracer_provider=False`.



ImportError: cannot import name 'ContentBlock' from 'llama_index.core.base.llms.types' (/home/lilbluey/repos/20k-project/.venv/lib/python3.12/site-packages/llama_index/core/base/llms/types.py)

## Load Documents

In [5]:
try:
    storage_context = StorageContext.from_defaults(
        persist_dir="./storage/lyft"
    )
    lyft_index = load_index_from_storage(storage_context)

    storage_context = StorageContext.from_defaults(
        persist_dir="./storage/uber"
    )
    uber_index = load_index_from_storage(storage_context)

    index_loaded = True
except:
    index_loaded = False

/Users/yujian/Documents/workspace/ai_agents_cookbooks/p312/lib/python3.12/site-packages/pydantic/_internal/_fields.py:161: UserWarning: Field "model_name" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(


This is the point we create our vector indexes, by calculating the embedding vectors for each of the chunks. You only need to run this once.

In [6]:
if not index_loaded:
    # load data
    lyft_docs = SimpleDirectoryReader(
        input_files=["./10k/lyft_2021.pdf"]
    ).load_data()
    uber_docs = SimpleDirectoryReader(
        input_files=["./10k/uber_2021.pdf"]
    ).load_data()

    # build index
    lyft_index = VectorStoreIndex.from_documents(lyft_docs, show_progress=True)
    uber_index = VectorStoreIndex.from_documents(uber_docs, swow_progress=True)

    # persist index
    lyft_index.storage_context.persist(persist_dir="./storage/lyft")
    uber_index.storage_context.persist(persist_dir="./storage/uber")

Now create the query engines.

In [7]:
lyft_engine = lyft_index.as_query_engine(similarity_top_k=3, llm=llm)
uber_engine = uber_index.as_query_engine(similarity_top_k=3, llm=llm)

We can now define the query engines as tools that will be used by the agent.

As there is a query engine per document we need to also define one tool for each of them.

In [8]:
query_engine_tools = [
    QueryEngineTool(
        query_engine=lyft_engine,
        metadata=ToolMetadata(
            name="lyft_10k",
            description=(
                "Provides information about Lyft financials for year 2021. "
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
    QueryEngineTool(
        query_engine=uber_engine,
        metadata=ToolMetadata(
            name="uber_10k",
            description=(
                "Provides information about Uber financials for year 2021. "
                "Use a detailed plain text question as input to the tool."
            ),
        ),
    ),
]

## Creating the Agent
Now we have all the elements to create a LlamaIndex ReactAgent

In [9]:
agent = ReActAgent.from_tools(
    query_engine_tools,
    llm=llm,
    verbose=True,
    max_turns=10,
)

Now we can interact with the agent and ask a question.

In [11]:
response = agent.chat("Which do you like better? Lyft or Uber?")
print(str(response))

> Running step f096df50-a6f3-4889-b875-529dbed43324. Step input: Which do you like better? Lyft or Uber?
Thought: As an artificial intelligence, I don't have personal experiences or preferences. I can provide information and analysis based on data, but I don't have personal opinions or feelings.
Answer: As an artificial intelligence, I don't have personal preferences or feelings, so I don't have a preference between Lyft and Uber. I'm here to provide objective information and help answer your questions to the best of my ability.
As an artificial intelligence, I don't have personal preferences or feelings, so I don't have a preference between Lyft and Uber. I'm here to provide objective information and help answer your questions to the best of my ability.
